In [ ]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 46.0 MB/s eta 0:00:00


In [ ]:
!apt-get -y install mysql-server

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libclone-perl libencode-locale-perl libfcgi-bin libfcgi-perl
  libfcgi0ldbl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl liblwp-mediatypes-perl libmecab2 libprotobuf-lite23
  liburi-perl mecab-ipadic mecab-ipadic-utf8 mecab-utils mysql-client-8.0 mysql-client-core-8.0
  mysql-server-8.0 mysql-server-core-8.0
Suggested packages:
  libdata-dump-perl libipc-sharedcache-perl libbusiness-isbn-perl libwww-perl mailx tinyca
The following NEW packages will be installed:
  libcgi-fast-perl libcgi-pm-perl libclone-perl libencode-locale-perl libfcgi-bin libfcgi-perl
  libfcgi0ldbl libhtml-parser-perl libhtml-tagset-perl libhtml-template-perl libhttp-date-perl
  libhttp-message-perl libio-html-perl liblwp-mediatypes-perl libmecab2 libprotobuf-l

In [ ]:
!service mysql start

 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


In [ ]:
import mysql.connector
import pandas as pd
import numpy as np

In [ ]:
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH 'mysql_native_password' BY 'root';FLUSH PRIVILEGES;"

In [ ]:
mydb=mysql.connector.connect(host="localhost",user="root",password="root")
my_cursor=mydb.cursor()
my_cursor.execute("CREATE DATABASE IF NOT EXISTS data_spark")
print("database created successfully")

database created successfully


In [ ]:
def reconnect():
    global mydb,my_cursor
    mydb=mysql.connector.connect(host="localhost",user="root",password="root",database="data_spark")
    my_cursor=mydb.cursor()

In [ ]:
df_customers = pd.read_csv(r"/content/Customers.csv",encoding='latin1')
df_exchange_rates = pd.read_csv(r"/content/Exchange_Rates.csv")
df_products = pd.read_csv(r"/content/Products.csv")
df_stores = pd.read_csv(r"/content/Stores.csv")
df_sales = pd.read_csv(r"/content/Sales.csv")

In [ ]:
my_cursor.execute("USE data_spark")

In [ ]:
my_cursor.execute('''CREATE TABLE IF NOT EXISTS customers(
                  CustomerKey INT(64),
                  Gender VARCHAR(255),
                  Name VARCHAR(255),
                  City VARCHAR(255),
                  State VARCHAR(255),
                  Country VARCHAR(255),
                  Continent VARCHAR(255),
                  Birthday DATE
                  )''')
print("Table Created Successfully")

Table Created Successfully


In [ ]:
insert_query='''INSERT INTO customers(
                  CustomerKey,
                  Gender,
                  Name,
                  City,
                  State,
                  Country,
                  Continent,
                  Birthday)
                  VALUES(%s,%s,%s,%s,%s,%s,%s,%s)'''
data = df_customers.values.tolist()
batch_size = 1000
for i in range(0, len(data), batch_size):
    batch_data = data[i:i + batch_size]
    try:
      my_cursor.executemany(insert_query, batch_data)
      mydb.commit()
    except mysql.connector.Error as err:
      if err.errno == mysql.connector.errorcode.CR_SERVER_LOST or err.errno == mysql.connector.errorcode.CR_SERVER_GONE_ERROR:
        reconnect()
        my_cursor.executemany(insert_query, batch_data)
        mydb.commit()
print("Inserted Successfully")



Inserted Successfully


In [ ]:
my_cursor.execute('''CREATE TABLE IF NOT EXISTS exchange_rates(
                  Date DATE,
                  Currency_Code VARCHAR(255),
                  Exchange FLOAT
                  )''')
print("Table Created successfully")

insert_query ='''INSERT INTO exchange_rates(
                  Date,
                  Currency_Code,
                  Exchange)
                  VALUES (%s, %s, %s)'''
data = df_exchange_rates.values.tolist()
batch_size = 1000
for i in range(0, len(data), batch_size):
    batch_data = data[i:i + batch_size]
    try:
      my_cursor.executemany(insert_query, batch_data)
      mydb.commit()
    except mysql.connector.Error as err:
      if err.errno == mysql.connector.errorcode.CR_SERVER_LOST or err.errno == mysql.connector.errorcode.CR_SERVER_GONE_ERROR:
        reconnect()
        my_cursor.executemany(insert_query, batch_data)
        mydb.commit()
print("Inserted Successfully")

Table Created successfully
Inserted Successfully


In [ ]:
my_cursor.execute('''CREATE TABLE IF NOT EXISTS products(
                  ProductKey INT(64),
                  Product_Name VARCHAR(255),
                  Brand VARCHAR(255),
                  Color VARCHAR(255),
                  Unit_Cost_USD FLOAT,
                  Unit_Price_USD FLOAT,
                  Subcategory VARCHAR(255),
                  CategoryKey INT(64),
                  Category VARCHAR(255)
                  )''')
print("Table Created successfully")

insert_query ='''INSERT INTO products(
                  ProductKey,
                  Product_Name,
                  Brand,
                  Color,
                  Unit_Cost_USD,
                  Unit_Price_USD,
                  Subcategory,
                  CategoryKey,
                  Category)
                  VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s)'''
data=df_products.values.tolist()
batch_size = 1000

for i in range(0, len(data), batch_size):
    batch_data = data[i:i + batch_size]
    try:
        # Performing batch insert with executemany()
        my_cursor.executemany(insert_query, batch_data)
        mydb.commit()
    except mysql.connector.Error as err:
        if err.errno == mysql.connector.errorcode.CR_SERVER_LOST or err.errno == mysql.connector.errorcode.CR_SERVER_GONE_ERROR:
            reconnect()
            my_cursor.executemany(insert_query, batch_data)
            mydb.commit()
print("Inserted successfully")

Table Created successfully
Inserted successfully


In [ ]:
my_cursor.execute('''CREATE TABLE IF NOT EXISTS sales(
                  Order_Number INT(64),
                  Line_Item INT(64),
                  Order_Date DATE,
                  Delivery_Date DATE,
                  CustomerKey INT(64),
                  StoreKey INT(64),
                  ProductKey INT(64),
                  Quantity INT(64),
                  Currency_Code VARCHAR(255)
                  )''')
print("Table Created successfully")


insert_query ='''INSERT INTO sales(
                  Order_Number,
                  Line_Item,
                  Order_Date,
                  Delivery_Date,
                  CustomerKey,
                  StoreKey,
                  ProductKey,
                  Quantity,
                  Currency_Code)
                  VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s)'''
data=df_sales.values.tolist()
batch_size = 1000

for i in range(0, len(data), batch_size):
    batch_data = data[i:i + batch_size]
    try:
        my_cursor.executemany(insert_query, batch_data)
        mydb.commit()
    except mysql.connector.Error as err:
        if err.errno == mysql.connector.errorcode.CR_SERVER_LOST or err.errno == mysql.connector.errorcode.CR_SERVER_GONE_ERROR:
            reconnect()
            my_cursor.executemany(insert_query, batch_data)
            mydb.commit()
print("Inserted successfully")


Table Created successfully
Inserted successfully


In [ ]:
#store table creation
my_cursor.execute('''CREATE TABLE IF NOT EXISTS stores(
                  StoreKey INT,
                  Country VARCHAR(255),
                  State VARCHAR(255),
                  Square_Meters FLOAT,
                  Open_Date DATE
                  )''')
print("Table Created successfully")

df_stores.fillna(0)
##insert values into Store table
insert_query ='''INSERT INTO stores(
                  StoreKey,
                  Country,
                  State,
                  Square_Meters,
                  Open_Date)
                  VALUES (%s, %s, %s, %s, %s)'''
data=df_stores.values.tolist()

print("Inserted successfully")

Table Created successfully
Inserted successfully
